In [60]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import plotly.express as px
def add_spread_data(df):
    df['spread'] = df.apply(lambda x: x['ask_price_1'] - x['bid_price_1'],axis = 1)
    print('here')
    df['mid'] = df.apply(lambda x: x['bid_price_1'] + (x['spread']/2),axis = 1)
    return df
def read_market_data(fileName):
    df = pd.read_csv(fileName).fillna("")
    cols = df.columns.tolist()[0].split(';')
    fin = pd.DataFrame(df.iloc[:,0].apply(lambda x: x.split(';')).values.tolist(),columns= cols)
    to_float_cols = ['day','timestamp','bid_price_1','bid_volume_1','bid_price_2','bid_volume_2','bid_price_3','bid_volume_3','ask_price_1','ask_volume_1','ask_price_2','ask_volume_2','ask_price_3','ask_volume_3','mid_price','profit_and_loss']
    fin[to_float_cols] =  fin[to_float_cols].applymap(lambda x: float(x) if x != '' else '')
    fin = add_spread_data(fin)
    return fin

def split_data_by_product(df):
    markets = dict()
    products  = df['product'].unique().tolist()    
    for prod in products:
        markets[prod]= df[df['product']==prod]
    return markets

def bollinger(df,col,window=10,deviations = 2):
    df[col+"_mean"] = df[col].rolling(window = window).mean()
    df[col+"_std"] = df[col].rolling(window = window).std() * 2
    df[col+"_hi"] = df[col+"_std"] + df[col+"_mean"]
    df[col+"_lo"] = df[col+"_mean"] - df[col+"_std"] 
    return df


def read_log_output(fname):
    logfile = open(fname,'r').readlines()
    logfile = [i.strip('\n') for i in logfile if ';' in i]
    logfile = [i[i.find('time'):] for  i in logfile]
    logfile = [i.split('|') for  i in logfile]
    logfile = [[item.split(';') for item in l] for l in logfile]
    logfile = [{item[0]:item[1] for item in l} for l in logfile]
    logdf = pd.DataFrame(logfile)
    return logdf

#market data
df = read_market_data('fd231b09-2b46-4a4a-a17c-83a0f28e7a53.csv')
markets = split_data_by_product(df)
# log file 
fname = 'fd231b09-2b46-4a4a-a17c-83a0f28e7a53.txt'
logdf = read_log_output(fname)
markets_td_df = split_data_by_product(logdf)
markets_td_df['BANANAS']['time'] = markets_td_df['BANANAS']['time'].astype(float)
markets_td_df['PEARLS']['time'] = markets_td_df['BANANAS']['time'].astype(float)


all_df = markets['BANANAS'].merge(markets_td_df['BANANAS'],how = 'left', left_on='timestamp', right_on='time')


here


In [232]:
import plotly.graph_objects as go 

from plotly.subplots import make_subplots

# fig = go.Figure()
cols_to_plot = ['mid_price','bid_price_1','bid_price_2','bid_price_3','ask_price_1','ask_price_2','ask_price_3','smart_price','smart_price_ask','smart_price_bid']
all_df[cols_to_plot] =  all_df[cols_to_plot].applymap(lambda x: float(x) if x != '' else '')
all_df['our_postion'] = all_df['our_postion'].astype(float)
df_cols = ['timestamp'] + cols_to_plot
plot_df = all_df.copy()

fig = make_subplots(rows=2, cols=1,shared_xaxes = True,row_heights = [.85,.15],vertical_spacing=0.01)
                    # specs=[[{}],[{"secondary_y": True}]])
for col in cols_to_plot:
    #line_shape='linear'))
    fig.add_trace(go.Scatter(x=plot_df['timestamp'],
                            y= plot_df[col],
                            name=col,mode='lines+markers',
                            text=col, ),
                            # secondary_y = False,
                            row =1,
                            col = 1) 
# fig.update_layout(
#     margin=dict(l=20, r=20, t=100, b=20)
# )

fig.add_trace(go.Scatter(
    x=plot_df['timestamp'],
    y= plot_df['our_postion'],
    name='our_position',
    # mode='lines+markers',
    line_shape='hv',
    text = 'our_position'),
    # secondary_y = True,
    row = 2,
    col = 1
)

fig.update_layout(
    autosize=False,
    width=1500,
    height=800)

# fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
# fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)
fig.show()


['day',
 'timestamp',
 'product_x',
 'bid_price_1',
 'bid_volume_1',
 'bid_price_2',
 'bid_volume_2',
 'bid_price_3',
 'bid_volume_3',
 'ask_price_1',
 'ask_volume_1',
 'ask_price_2',
 'ask_volume_2',
 'ask_price_3',
 'ask_volume_3',
 'mid_price',
 'profit_and_loss',
 'spread',
 'mid',
 'time',
 'product_y',
 'smart_price_bid',
 'smart_price',
 'smart_price_ask',
 'our_postion',
 ' buy_orders',
 ' sell_orders',
 ' our_previous_filled',
 ' market_previous_filled']